<a href="https://colab.research.google.com/github/siiseli19/StockTool/blob/master/StockTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock valuation tool project

Aim of this project is to create a tool for valuing stocks for personal use.

This tool can be used only for mature companies excluding financial institutions which have their own valuation methods.

# Setting up data pipeline

Creating a generic data pipeline for data fetch.

In [17]:
pip install edgartools

In [18]:
#Set up a generic data pipeline & getting the 10-K form
import requests
import pandas as pd
import edgar
from edgar import *
from edgar import set_identity
set_identity("Sami Ojala sami.ojala97@gmail.com")

headers = {'User-Agent': "sami.ojala97@gmail.com"}

def get_financials(ticker):

  tenk = Company(ticker).get_filings(form="10-K").latest(1).obj()
  financials = tenk.financials

  return financials



In [19]:
#get balance sheet
def get_balance_sheet(financials):
  balance_sheet = financials.balance_sheet
  return balance_sheet


In [20]:
#get cashflows
def get_cashflow_statement(financials):
  cashflow_statement = financials.cash_flow_statement
  return cashflow_statement


In [21]:
#get income statement
def get_income_statement(financials):
  income_statement = financials.income_statement
  return income_statement


# Company fundamentals & DCF inputs

Collecting all the needed fundamentals for DCF valuation.


In [22]:
#Get stock fundamentals

#form the growth rate for cashflow forecasts
#use damodarans approach, temporary % for now
def get_growth_rate():
  growth_rate = 1.05
  return get_growth_rate


#build EBIT
#calculate future cashflows
#returns CF array & growth factor
#inputs : financial statements df:s

#operating assets = cash + accounts receivable + inventory + prepaid expenses + fixed assets
#accounts payable + accrued operating expenses
# 2023 NWC - 2022 NWC
#CapEx = Ending PP&E – Beginning PP&E + Depreciation

#unlevered FCF = EBIT - Change in NWC - Capex
def get_cashflows(income_statement_df, balance_sheet_df, cashflow_statement_df):

  # Calculate EBIT for the year 2023
  gross_profit_2023 = income_statement_df.loc['GrossProfit', '2023-09-30']
  operating_expenses_2023 = income_statement_df.loc['OperatingExpenses', '2023-09-30']

  # EBIT = Gross Profit - Operating Expenses
  ebit_2023 = gross_profit_2023 - operating_expenses_2023

  return ebit_2023



#get terminal value
#use last forecasted cashflow
def get_terminal_value():

    # perpetual growth method
    exit_value = (future_cf5 * (1+perp_growth)) / (wacc-perp_growth)
    pass

# WACC

Building the weighted average cost of capita using CAPM. For cost of debt we will use Damodarans synthetic credit rating approach due to practical reasons.

### Cost of equity

In [23]:
#cost of equity using CAPM
def calculate_cost_of_equity():
    pass

### Cost of debt

In [24]:
def interest_coverage_and_RF(ticker):
    stock_ticker = ticker
    key = os.environ.get('API_KEY')

    # Interest coverage
    IS = requests.get(
        f'https://financialmodelingprep.com/api/v3/income-statement/{stock_ticker}?apikey={key}').json()  # income statement
    EBIT = IS[0]['ebitda'] - IS[0]['depreciationAndAmortization']  # index 0 for the most recent year
    interest_expense = IS[0]['interestExpense']
    interest_coverage_ratio = EBIT / interest_expense

    # RF
    start = datetime.datetime(2022, 1, 1)

    end = datetime.datetime.today().strftime('%Y-%m-%d')
    # end = datetime.datetime(2020, 7, 10)

    Treasury = web.DataReader(['TB1YR'], 'fred', start, end)
    RF = float(Treasury.iloc[-1])
    RF = RF / 100
    return [RF, interest_coverage_ratio]



#cost of debt for using Damodarans synthetic credit rating
def calculate_cost_of_debt(RF, interest_coverage_ratio):
    if interest_coverage_ratio > 8.5:
        # Rating is AAA
        credit_spread = 0.0063
    if (interest_coverage_ratio > 6.5) & (interest_coverage_ratio <= 8.5):
        # Rating is AA
        credit_spread = 0.0078
    if (interest_coverage_ratio > 5.5) & (interest_coverage_ratio <= 6.5):
        # Rating is A+
        credit_spread = 0.0098
    if (interest_coverage_ratio > 4.25) & (interest_coverage_ratio <= 5.49):
        # Rating is A
        credit_spread = 0.0108
    if (interest_coverage_ratio > 3) & (interest_coverage_ratio <= 4.25):
        # Rating is A-
        credit_spread = 0.0122
    if (interest_coverage_ratio > 2.5) & (interest_coverage_ratio <= 3):
        # Rating is BBB
        credit_spread = 0.0156
    if (interest_coverage_ratio > 2.25) & (interest_coverage_ratio <= 2.5):
        # Rating is BB+
        credit_spread = 0.02
    if (interest_coverage_ratio > 2) & (interest_coverage_ratio <= 2.25):
        # Rating is BB
        credit_spread = 0.0240
    if (interest_coverage_ratio > 1.75) & (interest_coverage_ratio <= 2):
        # Rating is B+
        credit_spread = 0.0351
    if (interest_coverage_ratio > 1.5) & (interest_coverage_ratio <= 1.75):
        # Rating is B
        credit_spread = 0.0421
    if (interest_coverage_ratio > 1.25) & (interest_coverage_ratio <= 1.5):
        # Rating is B-
        credit_spread = 0.0515
    if (interest_coverage_ratio > 0.8) & (interest_coverage_ratio <= 1.25):
        # Rating is CCC
        credit_spread = 0.0820
    if (interest_coverage_ratio > 0.65) & (interest_coverage_ratio <= 0.8):
        # Rating is CC
        credit_spread = 0.0864
    if (interest_coverage_ratio > 0.2) & (interest_coverage_ratio <= 0.65):
        # Rating is C
        credit_spread = 0.1134
    if interest_coverage_ratio <= 0.2:
        # Rating is D
        credit_spread = 0.1512

    cost_of_debt = RF + credit_spread
    return cost_of_debt


### WACC

In [25]:
#calculate WACC using cost of equity & debt
#debt/equity ratio
def calculate_wacc():
    pass

# DCF


Creating the DCF model. Forecasting future cashflows and discounting them back using WACC.


In [26]:
#cashflows come from EBIT

def forecast_cashflows_and_terminal_value(wacc, perp_growth_rate, forecasted_cashflows):

    # perpetual growth method
    exit_value = (last_future_cf5 * (1+perp_growth)) / (wacc-perp_growth)




    forecasted_cashflows_and_exit_value = []  # return

    #round
    forecasted_cashflows_and_exit_value.append(future_cf1)
    forecasted_cashflows_and_exit_value.append(future_cf2)
    forecasted_cashflows_and_exit_value.append(future_cf3)
    forecasted_cashflows_and_exit_value.append(future_cf4)
    forecasted_cashflows_and_exit_value.append(future_cf5)
    forecasted_cashflows_and_exit_value.append(exit_value)


    # return cashflows + terminal value
    return forecasted_cashflows_and_exit_value

In [27]:
def DCF():
    pass

    #Forecasts

    #Terminal value

    #Enterprise value = Sum of PVs

    #Equity value = + cash & equivalents, -debt

    #Intrinsic value of stock

# Trends

Identify trends and plot them

In [28]:
def doodoo():
  pass

# Run the program


Utilize above defined functions and create a DCF model for a specific stock

In [29]:
#driver function
import pandas as pd


ticker = 'AAPL'

financials = get_financials(ticker)

balance_sheet = get_balance_sheet(financials)
balance_sheet_df = balance_sheet.to_dataframe()

income_statement = get_income_statement(financials)
income_statement_df = income_statement.to_dataframe()

cashflow_statement = get_cashflow_statement(financials)
cashflow_statement_df = cashflow_statement.to_dataframe()





In [30]:
#print(balance_sheet_df)

In [31]:
print(income_statement)

                     Apple Inc. Consolidated Statement of Operations FY 2023                     
                                                                                                 
                                               2023-09-30         2022-09-24         2021-09-25  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  SALES:                                                                                         
  Total Net Sales                         383,285,000,000    394,328,000,000    365,817,000,000  
                                                                                                 
  COST OF SALES:                                                                                 
  Cost Goods and Services Sold            214,137,000,000    223,546,000,000    212,981,000,000  
                                                                                                 
  Gross Profit      

In [32]:
print(cashflow_statement)

                                     Apple Inc. Cashflow Statement FY 2023                                      
                                                                                                                
                                                              2023-09-30         2022-09-24         2021-09-25  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  OPERATING ACTIVITIES:                                                                                         
    Net Income                                            96,995,000,000     99,803,000,000     94,680,000,000  
    Depreciation and Amortization                         11,519,000,000     11,104,000,000     11,284,000,000  
    Stock-based Compensation                              10,833,000,000      9,038,000,000      7,906,000,000  
    Other Noncash Income/Expense                           2,227,000,000     -1,006,000,000     

In [33]:
print(balance_sheet)

                         Apple Inc. Balance Sheet FY 2023                          
                                                                                   
                                                    2023-09-30         2022-09-24  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  ASSETS:                                                                          
  CURRENT ASSETS:                                                                  
    Cash and Cash Equivalents                   29,965,000,000     23,646,000,000  
    Marketable Securities                       31,590,000,000     24,658,000,000  
    Accounts Receivable, net                    29,508,000,000     28,184,000,000  
    Vendor non-trade Receivables                31,477,000,000     32,748,000,000  
    Inventories                                  6,331,000,000      4,946,000,000  
    Other Current Assets                        14,695,000,000     21,223,00

In [34]:
#print(cashflow_statement_df)

In [35]:
# Example usage
print(income_statement_df.loc['RevenueFromContractWithCustomerExcludingAssessedTax', '2023-09-30'])
gross_profit_2022 = income_statement_df.loc['GrossProfit', '2022-09-24']
print(gross_profit_2022)

383285000000
170782000000


# Plan / Guidelines

PRELIMINARY PLAN

1. Scrape needed data and make a generic pipeline
1.1 Store data into pandas dataframe for quick use
2. Build a DCF model
3. Sensitivity analysis with different parameters
4. Make an Excel file of the output for comparing
4.1 Make plots to spot trends
  - debt / equity
  - earnings
  - FCF
  - Margins
  - ROIC
5. Make a screener & apply suitable algorithms to sort data
6. Run above pipeline in Google cloud
7. Send email for potential buy opportunity
8. Transformer for 10-K summaries


'''
Having the CIK number of a company of interest,
one is now able to query the financial data reported by that company,
using the companyconcept path of the SEC API [8] and the following HTTP scheme:
https://data.sec.gov/api/xbrl/companyconcept/[CIK]/us-gaap/[Tag].json, where:

CIK is the CIK number of the company,
Tag is a tag from the US-GAAP taxonomy.
The US-GAAP taxonomy is a collection of eXtensible Business Reporting (XBRL) tags used by
companies for labeling financial data and reporting it to the SEC.

filings = get_filings(form="10-K")

filings = get_filings(year=range(2010, 2020)
